# Importing a new time series

This notebook shows how to add a time series which will determine the grid price at every time step, overwriting the constant value in the base model.  
We change the cost function, similar to [the subsidy example](ModifyExistingConstraint.ipynb).

In [1]:
from pyehub.energy_hub.ehub_model import EHubModel
from pyehub.energy_hub.utils import constraint
from pyehub.outputter import pretty_print

Extend the `EHubModel` class and alter the constraint which calculates the operating cost.
+ `calculate_operating_cost(self)` is a constraint in the `EHubModel` class which calculates the operating cost using a constant grid price in the input file.
+ We also include Feed In Tariff income generated by exporting energy from `export_streams`.
+ Specify the `grid_price` time series.
+ For every converter which has 'Grid' as input, we calculate the operating cost using the `grid_price` time series. For other input streams, we find the total energy imported across all time steps and multiply it by a constant `FUEL_PRICE` of that input stream in the usual way.
+ Impose the operating cost constraint (i.e. `operating_cost = total_fuel_bill - total_export_income`).

In [2]:
class MyModel(EHubModel):
    """
    This is a subclass of the original EHubModel.

    We create a subclass so that we can modify the superclass (or parent
    class) without modifying the superclass's source code.
    """

    @constraint()
    def calc_operating_cost(self):
        """
        Calculate a new total cost that uses a price per time step.

        Instead of using a fixed price for electricity from the grid, we
        instead use a time series that contains the price per time step.
        """
        # calculate the export income
        total_export_income = 0
        
        for stream in self.export_streams:
            # If the price is time series
            if self.FEED_IN_TARIFFS[stream] in self.TIME_SERIES:
                for t in self.time:
                    price = self.TIME_SERIES[self.FEED_IN_TARIFFS[stream]][t]
                    energy_exported = self.energy_exported[t][stream]
                    total_export_income += price * energy_exported

            else:
                total_energy_exported = sum(self.energy_exported[time][stream]
                                            for time in self.time)
                price = self.FEED_IN_TARIFFS[stream]
                total_export_income += price * total_energy_exported
        
        # Here we add the specific time series called `grid_price`
        grid_price = {0: 0.13,
                      1: 1.13,
                      2: 2.13,
                      3: 0.13,
                      4: 0.13,
                      5: 0.13,
                      6: 0.13,
                      7: 0.13,
                      8: 0.13,
                      9: 0.13,
                      10: 0.13}
        total_fuel_bill = 0
        for tech in self._data.converters:
            
            if 'Grid' in tech.inputs:
                total_fuel_bill += sum(grid_price[t]
                                      * self.energy_imported[t]['Grid']
                                      for t in self.time)

            # Default to the fixed price for non `Grid` converters
            for inp in tech.inputs:
                if (inp != 'Grid') and (inp in self.import_streams):
                    total_fuel_bill += self.FUEL_PRICES[inp]*sum(self.energy_imported[t][inp] for t in self.time)

        
        return self.operating_cost == total_fuel_bill - total_export_income

In [3]:
excel_file = 'test_file_all_constraints_work.xlsx' # name of the excel file. [This must be in the current directory]
my_model = MyModel(excel=excel_file) # instantiate our model. Nothing is solved at this point.
results = my_model.solve() # solve the model and get back our results
pretty_print(results) # print the results to the console

Version: 0.1.0
Solver
	termination condition: Optimal
	time: 0.035978727999463445
Solution

========== Stuff ==========

ANNUAL_MAINTENANCE_STORAGE_COSTS: 
                ANNUAL_MAINTENANCE_STORAGE_COSTS
Battery                                        0
Hot Water Tank                                 0

BIG_M: 
99999

Battery: 
0.0

Boiler: 
10.0

CARBON_CREDITS: 
            CARBON_CREDITS
Elec                      
Gas                       
Grid                      
Heat                      
Irradiation               
PV_Elec                0.3

CARBON_FACTORS: 
            CARBON_FACTORS
Elec                      
Gas                  0.194
Grid                  0.35
Heat                      
Irradiation               
PV_Elec                   

CHARGING_EFFICIENCY: 
                CHARGING_EFFICIENCY
Battery                        0.99
Hot Water Tank                 0.99

CHP: 
5.0

CONVERSION_EFFICIENCY: 
          Elec  ...  PV_Elec
Boiler     0.0  ...    0.000
CHP        0.